In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""Display some predictions."""
import os
from pathlib import Path
import numpy as np
from tensorflow.keras.models import load_model
# import matplotlib.pyplot as plt
from nmp import dataset
# from tensorflow.keras.layers import LSTM
import tensorflow as tf
import copy
import pandas as pd
from nmp.dataset import pyplot_piano_roll, write_midi
import matplotlib.pyplot as plt
import time
import pypianoroll
import random

P = Path(os.path.abspath(''))  # Compatible with Jupyter Notebook

PLOTS = P / 'plots'  # Plots path
BS = 64
FS = 24  # Sampling frequency. 10 Hz = 100 ms
Q = 0  # Quantize?
st = 10  # Past timesteps
num_ts = 10  # Predicted timesteps
DOWN = 12  # Downsampling factor
D = "data/Nottingham/test"  # Dataset (synth or data)

MODEL = 'simpleRNN-nottingham.h5'
# MODEL = 'ff-z-nottingham'
model_type = 'rnn'

LOW_LIM = 33  # A1
HIGH_LIM = 97  # C7

NUM_NOTES = HIGH_LIM - LOW_LIM
CROP = [LOW_LIM, HIGH_LIM]  # Crop plots

# TensorFlow stuff
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# Load model.
model = load_model(filepath=str(P / 'models' / MODEL),
                   custom_objects=None, compile=True)
model.summary()

tempo = 138

# midi-test





In [ ]:
# def fill_gap(position, size)
#     start = position - 120
#     end = position

#     past = dataset.downsample_roll(test_file[start:end, :], 10, 12)
#     past = np.array([past])

#     predictions = model.predict(past)
#     predictions_bin = dataset.ranked_threshold(predictions, steps=10, how_many=5)
#     predictions_bin = predictions_bin.reshape((10, 64))
#     # pyplot_piano_roll(predictions_bin, low_lim=+LOW_LIM, high_lim=HIGH_LIM)

#     upsampled = copy.deepcopy(predictions_bin)
#     upsampled = dataset.upsample_roll(upsampled, 10, 12)
#     # pyplot_piano_roll(arr, low_lim=+LOW_LIM, high_lim=HIGH_LIM)

#     final = copy.deepcopy(test_file)
#     final[end: end+12*size] = upsampled[:12*size]
    
#     return final
# #     pyplot_piano_roll(final, low_lim=+LOW_LIM, high_lim=HIGH_LIM)

In [ ]:
FILE = 'ashover_simple_chords_5.mid'
BASE = 0

import importlib
importlib.reload(dataset)
test_file = dataset.import_one(str(D + '/' + FILE), 24)
test_file = test_file[:, LOW_LIM:HIGH_LIM]
pyplot_piano_roll(test_file, low_lim=+LOW_LIM, high_lim=HIGH_LIM)
STOP = 960

In [ ]:
print(test_file.shape)
random.seed(42)
before = copy.deepcopy(test_file)
err_pos = range(10, int(test_file.shape[0]/12))
err_pos = [x*12 for x in err_pos]
# for error in [60*x for x in range(1, 50)]:
for error in err_pos:
    occurs = random.choice([1, 0, 0, 0, 0, 0, 0])
    if occurs:
        print
        print("Error at timestep n. %d" % error)
        try:
            if model_type == 'ff':
                test_file = dataset.fill_gap(test_file, model, error, size=1, baseline=BASE, how_many=4)
            elif model_type == 'rnn':
                test_file = dataset.fill_gap_rnn(test_file, model, error, baseline=BASE, how_many=4)
        except Exception:
            break
print(test_file.shape)

In [ ]:
pyplot_piano_roll(before[0:200], low_lim=+LOW_LIM, high_lim=HIGH_LIM, db=[60])
plt.ylim([33, 97])
pyplot_piano_roll(test_file[0:200], low_lim=+LOW_LIM, high_lim=HIGH_LIM, db=[60])
plt.ylim([33, 97])

In [ ]:
pyplot_piano_roll(test_file, low_lim=+LOW_LIM, high_lim=HIGH_LIM)

f0 = copy.deepcopy(test_file)
dataset.write_midi(f0, str(P / 'audio_output' / (FILE + '_filled_rnn.mid')),
                   LOW_LIM, HIGH_LIM, tempo=tempo, br=24)

In [ ]:
# end = STOP + 120

# past = dataset.downsample_roll(test_file[STOP:end, :], 10, 12)
# past = np.array([past])

# predictions = model.predict(past)
# predictions_bin = dataset.ranked_threshold(predictions, steps=10, how_many=5)
# predictions_bin = predictions_bin.reshape((10, 64))
# # pyplot_piano_roll(predictions_bin, low_lim=+LOW_LIM, high_lim=HIGH_LIM)

# upsampled = copy.deepcopy(predictions_bin)
# upsampled = dataset.upsample_roll(upsampled, 10, 12)
# # pyplot_piano_roll(arr, low_lim=+LOW_LIM, high_lim=HIGH_LIM)

# final = copy.deepcopy(test_file)
# final[end: end+12*5] = upsampled[:12*5]
